In [1]:
import neuroglancer
# import cloudvolume

In [2]:
import numpy as np
from collections import OrderedDict
import webbrowser

In [3]:
neuroglancer.set_static_content_source(url='https://neuromancer-seung-import.appspot.com')

In [4]:
viewer = neuroglancer.Viewer()
# This volume handle can be used to notify the viewer that the data has changed.

with viewer.txn() as s:
    s.layers['segmentation'] = neuroglancer.SegmentationLayer(source='precomputed://http://localhost:1339'
    )

print(viewer)

http://127.0.0.1:62925/v/ee832e762844eaa62abce392e4d39b86a6e4dcff/


Now let's use the actual entire JSON dict to get all segment ids.

I downloaded the Princeton Mouse CSV file to, so I can now make a dict out of the two columns of interest, "id" and "name"

In [6]:
data_dir = '/Users/athair/Git2/lightserv_dj_demo/data'
csv_file = data_dir + '/Princeton_mouse_id_table.csv'
import csv

with open(csv_file, mode='r') as infile:
        reader = csv.reader(infile)
        atlas_dict = {}
        next(reader) # skips the header
        for row in reader:
            name,id = row[1],row[3]
            atlas_dict[int(id)]=name

In [7]:
# get the message to print out the value in the atlas dictionary, which is the region name
num_actions = 0
def my_action3(s):
    global num_actions
    num_actions += 1
    with viewer.config_state.txn() as st:
        region_id = s.selected_values['segmentation']
        region_name = atlas_dict.get(region_id)
        st.status_messages['hello'] = ('%i:%s' %
                                     (region_id, region_name))
    print('Got my-action %i' % num_actions)
#     print('  Mouse position: %s' % (s.mouse_voxel_coordinates,))
    print('  Layer selected values: %s' % (s.selected_values,))
viewer.actions.add('my-action3', my_action3)
with viewer.config_state.txn() as s:
    s.input_event_bindings.viewer['keyp'] = 'my-action3'
    s.status_messages['hello'] = 'Welcome to the segment labeling example. Press "p" to see region name under cursor.'



Got my-action 1
  Layer selected values: Map({"segmentation": 362})
Got my-action 2
  Layer selected values: Map({"segmentation": 581})
Got my-action 3
  Layer selected values: Map({"segmentation": 935})


In [9]:
ss = neuroglancer.ScreenshotSaver(viewer, data_dir+'/atlas_key_binding_example.png')
ss.capture()  

KeyboardInterrupt: 